In [ ]:
! pip install -q kaggle

In [ ]:
!mkdir ~/kaggle

mkdir: cannot create directory ‘/root/kaggle’: File exists


In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [1]:
# Create directories to store the training, testing and validation data respectively
!mkdir train
!mkdir test
!mkdir valid


In [ ]:
# Downloading the RLVS dataset
!kaggle datasets download mohamedmustafa/real-life-violence-situations-dataset -p /content/ --unzip

100% 3.57G/3.58G [00:27<00:00, 157MB/s]
100% 3.58G/3.58G [00:27<00:00, 140MB/s]


In [ ]:
# Get the video names of violence and non-violence
import os
non_violent=os.listdir("/content/Real Life Violence Dataset/NonViolence")
violent=os.listdir("/content/Real Life Violence Dataset/Violence")


In [ ]:
# Splitting the entire dataset into 8:1:1 for train:validation:test
import random
def split_test_train(lis):
  n=len(lis)
  other=random.sample(range(n),int(n/10))
  return ([x for i,x in enumerate(lis) if not i in other],[x for i,x in enumerate(lis) if i in other])

def split_train_valid(lis):
  n=len(lis)
  other=random.sample(range(n),int(n/9))
  return ([x for i,x in enumerate(lis) if not i in other],[x for i,x in enumerate(lis) if i in other])

train_non_violent,test_non_violent=split_test_train(non_violent)
train_violent,test_violent=split_test_train(violent)

train_non_violent,valid_non_violent=split_train_valid(train_non_violent)
train_violent,valid_violent=split_train_valid(train_violent)

In [ ]:
# Moving the videos into the appropriate folder based on the split
import shutil
src_path="/content/Real Life Violence Dataset/NonViolence/"
for filename in train_non_violent:
  shutil.move(os.path.join(src_path,filename), "/content/train")
for filename in test_non_violent:
  shutil.move(os.path.join(src_path,filename), "/content/test")
for filename in valid_non_violent:
  shutil.move(os.path.join(src_path,filename), "/content/valid")


src_path="/content/Real Life Violence Dataset/Violence/"
for filename in train_violent:
  shutil.move(os.path.join(src_path,filename), "/content/train")
for filename in test_violent:
  shutil.move(os.path.join(src_path,filename), "/content/test")
for filename in valid_violent:
  shutil.move(os.path.join(src_path,filename), "/content/valid")

In [ ]:
import pandas as pd
train=[]
test=[]
valid=[]
for filename in train_non_violent:
  train.append([filename,"Non Violent"])
for filename in train_violent:
  train.append([filename,"Violent"])
for filename in test_non_violent:
  test.append([filename,"Non Violent"])
for filename in test_violent:
  test.append([filename,"Violent"])
for filename in valid_non_violent:
  valid.append([filename,"Non Violent"])
for filename in valid_violent:
  valid.append([filename,"Violent"])

# Store a dataframe storing the video name and its label(violent or non-violent)
train_df = pd.DataFrame(train, columns =["video_name","tag"])
test_df = pd.DataFrame(test, columns =["video_name","tag"])
valid_df=pd.DataFrame(valid, columns =["video_name","tag"])

In [ ]:
# Randomly shuffle them
train_df=train_df.sample(frac = 1)
test_df=test_df.sample(frac = 1)
valid_df=valid_df.sample(frac = 1)

In [4]:
from tensorflow.keras import layers
from tensorflow import keras
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os
import io
import ipywidgets

In [ ]:
MAX_SEQ_LENGTH = 30 # The maximum number of frames extracted from each video
IMG_SIZE = 90 # the width/lenght of each frame extracted

In [ ]:
# Layer for cropping the frame to the desired size
center_crop_layer = layers.CenterCrop(IMG_SIZE,IMG_SIZE)

def crop_center(frame):
    '''Crops the frame and outputs a numpy array'''
    cropped = center_crop_layer(frame[None, ...])
    cropped = cropped.numpy().squeeze()
    return cropped

def load_video(path, max_frames):
    '''Extracts frames at fixed intervals throught the video till the video ends or the max_seq_lenght is reached'''
    frames_to_take_per_sec=5
    cap = cv2.VideoCapture(path)
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frames = []
    
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    k=int(fps/frames_to_take_per_sec)
    i=0
    j=0
    try:
      while j<max_frames:
        ret, frame = cap.read()
        if not ret:
          break
        if i%k==0:
          j+=1
          frame= crop_center(frame)
          frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
          frames.append(frame)
        i+=1
    finally:
      cap.release()
      return np.array(frames[:max_frames])





# Generates the label for the set of frames
label_processor = keras.layers.StringLookup(
    num_oov_indices=0, vocabulary=np.unique(train_df["tag"]), mask_token=None
)



def prepare_all_videos(df, root_dir):
    '''Converts all the videos in the folder to the desired max_seq_lenght and img_size 
    along with their labels for further processing'''
    num_samples = len(df)
    video_paths = df["video_name"].values.tolist()
    labels = df["tag"].values
    labels = label_processor(labels).numpy()

    videos = np.zeros(
        shape=(num_samples, MAX_SEQ_LENGTH, IMG_SIZE,IMG_SIZE), dtype="float32"
    )

    # For each video.
    for idx, path in enumerate(video_paths):
        # Gather all its frames and add a batch dimension.
        frames = load_video(os.path.join(root_dir, path),MAX_SEQ_LENGTH)

        # Pad shorter videos.
        if len(frames) < MAX_SEQ_LENGTH:
            diff = MAX_SEQ_LENGTH - len(frames)
            padding = np.empty((diff, IMG_SIZE, IMG_SIZE), dtype="float32")
            for d in range(diff):
              padding[d:]=frames[-1:]
            frames = np.concatenate((frames, padding))

        videos[idx,] = frames

    return videos, labels



Total videos for training: 1600
Total videos for testing: 200
Total videos for validating: 200
['Non Violent', 'Violent']


In [ ]:
train_videos, train_labels = prepare_all_videos(train_df,"/content/train")

In [ ]:
test_videos, test_labels = prepare_all_videos(test_df,"/content/test") 

In [ ]:
valid_videos, valid_labels = prepare_all_videos(valid_df,"/content/valid") 

In [ ]:
BATCH_SIZE = 32
AUTO = tf.data.AUTOTUNE
INPUT_SHAPE = (MAX_SEQ_LENGTH, IMG_SIZE, IMG_SIZE, 1)
NUM_CLASSES = 1

# OPTIMIZER
LEARNING_RATE = 1e-4

# TRAINING
EPOCHS = 40

# TUBELET EMBEDDING
PATCH_SIZE = (8, 8, 8)
NUM_PATCHES = (INPUT_SHAPE[0] // PATCH_SIZE[0]) ** 2

# ViViT ARCHITECTURE
LAYER_NORM_EPS = 1e-6
PROJECTION_DIM = 128
NUM_HEADS = 8
NUM_LAYERS = 8

In [ ]:

@tf.function
def preprocess(frames: tf.Tensor, label: tf.Tensor):
    """Preprocess the frames tensors and parse the labels."""
    # Preprocess images
    frames = tf.image.convert_image_dtype(
        frames[
            ..., tf.newaxis
        ],  # The new axis is to help for further processing with Conv3D layers
        tf.float32,
    )
    # Parse label
    label = tf.cast(label, tf.float32)
    return frames, label


def prepare_dataloader(
    videos: np.ndarray,
    labels: np.ndarray,
    loader_type: str = "train",
    batch_size: int = BATCH_SIZE,
):
    """Utility function to prepare the dataloader."""
    dataset = tf.data.Dataset.from_tensor_slices((videos, labels))

    if loader_type == "train":
        dataset = dataset.shuffle(BATCH_SIZE * 2)

    dataloader = (
        dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
        .batch(batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )
    return dataloader


trainloader = prepare_dataloader(train_videos, train_labels, "train")
train_videos=[] 
train_labels=[]

validloader = prepare_dataloader(valid_videos, valid_labels, "valid")
valid_videos=[] 
valid_labels=[]

testloader = prepare_dataloader(test_videos, test_labels, "test")
test_videos=[] 
test_labels=[]


In [ ]:

class TubeletEmbedding(layers.Layer):
    def __init__(self, embed_dim, patch_size, **kwargs):
        super().__init__(**kwargs)
        self.projection = layers.Conv3D(
            filters=embed_dim,
            kernel_size=patch_size,
            strides=patch_size,
            padding="VALID",
        )
        self.flatten = layers.Reshape(target_shape=(-1, embed_dim))

    def call(self, videos):
        projected_patches = self.projection(videos)
        flattened_patches = self.flatten(projected_patches)
        return flattened_patches


In [ ]:

class PositionalEncoder(layers.Layer):
    def __init__(self, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim

    def build(self, input_shape):
        _, num_tokens, _ = input_shape
        self.position_embedding = layers.Embedding(
            input_dim=num_tokens, output_dim=self.embed_dim
        )
        self.positions = tf.range(start=0, limit=num_tokens, delta=1)

    def call(self, encoded_tokens):
        # Encode the positions and add it to the encoded tokens
        encoded_positions = self.position_embedding(self.positions)
        encoded_tokens = encoded_tokens + encoded_positions
        return encoded_tokens


In [ ]:

def create_vivit_classifier(
    tubelet_embedder,
    positional_encoder,
    input_shape=INPUT_SHAPE,
    transformer_layers=NUM_LAYERS,
    num_heads=NUM_HEADS,
    embed_dim=PROJECTION_DIM,
    layer_norm_eps=LAYER_NORM_EPS,
    num_classes=NUM_CLASSES,
):
    # Get the input layer
    inputs = layers.Input(shape=input_shape)
    # Create patches.
    patches = tubelet_embedder(inputs)
    # Encode patches.
    encoded_patches = positional_encoder(patches)

    # Create multiple layers of the Transformer block.
    for _ in range(transformer_layers):
        # Layer normalization and MHSA
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim // num_heads, dropout=0.1
        )(x1, x1)

        # Skip connection
        x2 = layers.Add()([attention_output, encoded_patches])

        # Layer Normalization and MLP
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        x3 = keras.Sequential(
            [
                layers.Dense(units=embed_dim * 4, activation=tf.nn.relu),
                layers.Dense(units=embed_dim, activation=tf.nn.relu),
            ]
        )(x3)

        # Skip connection
        encoded_patches = layers.Add()([x3, x2])

    # Layer normalization and Global average pooling.
    representation = layers.LayerNormalization(epsilon=layer_norm_eps)(encoded_patches)
    representation = layers.GlobalAvgPool1D()(representation)

    # Classify outputs.
    outputs = layers.Dense(units=num_classes, activation="sigmoid")(representation)

    # Create the Keras model.
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


In [ ]:

def run_experiment():
    # Initialize model
    model = create_vivit_classifier(
        tubelet_embedder=TubeletEmbedding(
            embed_dim=PROJECTION_DIM, patch_size=PATCH_SIZE
        ),
        positional_encoder=PositionalEncoder(embed_dim=PROJECTION_DIM),
    )

    # Compile the model with the optimizer, loss function
    # and the metrics.
    optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
    model.compile(
        optimizer=optimizer,
        loss="binary_crossentropy",
        metrics=[
            keras.metrics.BinaryAccuracy(name="accuracy"),
        ],
    )

    # Train the model.
    _ = model.fit(trainloader, epochs=EPOCHS, validation_data=validloader)

    _, accuracy = model.evaluate(testloader)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return model


model = run_experiment()

Epoch 1/40
50/50 [==============================] - 30s 303ms/step - loss: 0.8171 - accuracy: 0.4888 - val_loss: 0.6852 - val_accuracy: 0.5400
Epoch 2/40
50/50 [==============================] - 14s 283ms/step - loss: 0.7081 - accuracy: 0.4869 - val_loss: 0.6863 - val_accuracy: 0.5050
Epoch 3/40
50/50 [==============================] - 14s 282ms/step - loss: 0.7021 - accuracy: 0.4825 - val_loss: 0.6967 - val_accuracy: 0.5000
Epoch 4/40
50/50 [==============================] - 14s 283ms/step - loss: 0.7016 - accuracy: 0.5169 - val_loss: 0.6776 - val_accuracy: 0.5150
Epoch 5/40
50/50 [==============================] - 14s 284ms/step - loss: 0.6908 - accuracy: 0.5113 - val_loss: 0.6756 - val_accuracy: 0.5450
Epoch 6/40
50/50 [==============================] - 14s 285ms/step - loss: 0.6841 - accuracy: 0.5419 - val_loss: 0.6750 - val_accuracy: 0.6100
Epoch 7/40
50/50 [==============================] - 14s 288ms/step - loss: 0.6928 - accuracy: 0.5125 - val_loss: 0.6870 - val_accuracy: 0.5250

In [ ]:

def prepare_single_video(frames):
    '''Outputs the video in the desired img_size and seq_lenght'''
    if len(frames) < MAX_SEQ_LENGTH:
            diff = MAX_SEQ_LENGTH - len(frames)
            padding = np.empty((diff, IMG_SIZE, IMG_SIZE), dtype="float32")
            for d in range(diff):
              padding[d:]=frames[-1:]
            frames = np.concatenate((frames, padding))


    return frames


def predict_action(path):
    '''Uses the model to predict whether the video has violence or not'''
    class_vocab = label_processor.get_vocabulary()
    frames = load_video(path,MAX_SEQ_LENGTH)
    video = prepare_single_video(frames)
    pred = model.predict(tf.expand_dims(video, axis=0))[0]
    if pred>0.5:
      return class_vocab[1]
    else:
      return class_vocab[0]




In [ ]:
# Checking the accuracy of the above model on occluded videos
v=0
for vid in os.listdir("/content/drive/MyDrive/Capstone Resources/data/vidsWithObstructions"):
  if predict_action("/content/drive/MyDrive/Capstone Resources/data/vidsWithObstructions/"+vid)=="Violent":
    v+=1
print("Accuracy on videos with occlusions",round(v*100/len(os.listdir("/content/drive/MyDrive/Capstone Resources/data/vidsWithObstructions")),2),"%")

1/1 [==============================] - 0s 34ms/step
Accuracy on videos with occlusions 95.12 %


In [ ]:
# Checking the accuracy of the above model on collected indoor violence videos
v=0
for vid in os.listdir("/content/drive/MyDrive/Capstone Resources/Phase 1/Videos/Clips/"):
  if predict_action("/content/drive/MyDrive/Capstone Resources/Phase 1/Videos/Clips/"+vid)=="Violent":
    v+=1
print("Accuracy on videos with clips",round(v*100/len(os.listdir("/content/drive/MyDrive/Capstone Resources/Phase 1/Videos/Clips/")),2),"%")

